In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('dirty_cafe_sales.csv')

data.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [3]:
# Replace ERROR and UNKNOWN values with NaN in all columns
data = data.replace(['ERROR', 'UNKNOWN'], np.nan)

In [24]:
data.describe()

,Quantity,Price Per Unit,Total Spent,Transaction Date
count,8564.000000,8564.000000,8564.000000,8564
mean,3.025222,2.952826,8.945061,2023-07-02 06:00:30.266230784
min,1.000000,1.000000,1.000000,2023-01-01 00:00:00
25%,2.000000,2.000000,4.000000,2023-04-01 00:00:00
50%,3.000000,3.000000,8.000000,2023-07-02 00:00:00
75%,4.000000,4.000000,12.000000,2023-10-03 00:00:00
max,5.000000,5.000000,25.000000,2023-12-31 00:00:00
std,1.423127,1.278280,6.016525,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9031 non-null   object
 2   Quantity          9521 non-null   object
 3   Price Per Unit    9467 non-null   object
 4   Total Spent       9498 non-null   object
 5   Payment Method    6822 non-null   object
 6   Location          6039 non-null   object
 7   Transaction Date  9540 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


Kita mengubah dtype yang kurang tepat dan menambahkan kolumn day ya menandakan hari apa dia memesan

In [5]:
# Convert numeric columns to float
data['Price Per Unit'] = pd.to_numeric(data['Price Per Unit'], errors='coerce')
data['Total Spent'] = pd.to_numeric(data['Total Spent'], errors='coerce')
data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')

# Convert Transaction Date to datetime if not already
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])

# Extract day of week and hour for time-based analysis
data['Day of Week'] = data['Transaction Date'].dt.day_name()
# data['Hour'] = data['Transaction Date'].dt.hour

# Print new data types
print(data.dtypes)

Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
Day of Week                 object
dtype: object


Identifikasi Missing Values

In [6]:
# Count remaining null values in each column
print("\nNull values count in each column:")
print(data.isnull().sum())


Null values count in each column:
Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
Day of Week          460
dtype: int64


Penanganan Nilai Null

- Pada kolom **Item**, baris dengan nilai *Null* akan **dihapus**, karena tanpa informasi produk, analisis tidak dapat dilakukan.

- Untuk kolom **Quantity**, **Price per Unit**, dan **Total Spent**:
  - Jika **salah satu** dari ketiga kolom tersebut bernilai *Null*, maka nilainya dapat **dihitung menggunakan logika matematika sederhana**:
    ```
    Quantity × Price per Unit = Total Spent
    ```
    Selama dua dari tiga nilai tersedia, nilai yang hilang bisa dihitung.
  - Namun, jika sebuah baris memiliki **dua atau lebih nilai *Null*** dari ketiga kolom tersebut, maka baris tersebut akan **dihapus** karena tidak memungkinkan untuk direkonstruksi.



In [7]:
data = data.dropna(subset=['Item']) # Drop rows where 'Item' is NaN
# Display the first few rows after dropping NaN values
print("Dataset after dropping rows where Item is NaN:")
display(data.head(10))

Dataset after dropping rows where Item is NaN:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Day of Week
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08,Friday
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16,Tuesday
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19,Wednesday
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27,Thursday
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11,Sunday
5,TXN_2602893,Smoothie,5.0,4.0,20.0,Credit Card,NaN,2023-03-31,Friday
7,TXN_6699534,Sandwich,4.0,4.0,16.0,Cash,NaN,2023-10-28,Saturday
9,TXN_2064365,Sandwich,5.0,4.0,20.0,NaN,In-store,2023-12-31,Sunday
10,TXN_2548360,Salad,5.0,5.0,25.0,Cash,Takeaway,2023-11-07,Tuesday
11,TXN_3051279,Sandwich,2.0,4.0,8.0,Credit Card,Takeaway,NaT,NaN


In [8]:
mask = data['Quantity'].isna() & data['Total Spent'].notna() & data['Price Per Unit'].notna() & (data['Price Per Unit'] > 0)
data.loc[mask, 'Quantity'] = (data.loc[mask, 'Total Spent'] / data.loc[mask, 'Price Per Unit']).round()

# Display the number of remaining NaN values in Quantity
print(f"NaN values in Quantity column after filling: {data['Quantity'].isna().sum()}")

# Show a few examples of fixed rows
print("\nSample of rows where Quantity was calculated:")
display(data[mask].head())

NaN values in Quantity column after filling: 35

Sample of rows where Quantity was calculated:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Day of Week
20,TXN_3522028,Smoothie,5.0,4.0,20.0,Cash,In-store,2023-04-04,Tuesday
55,TXN_5522862,Cookie,2.0,1.0,2.0,Credit Card,Takeaway,2023-03-19,Sunday
57,TXN_2080895,Cake,1.0,3.0,3.0,Digital Wallet,In-store,2023-04-19,Wednesday
66,TXN_8501819,Juice,2.0,3.0,6.0,Cash,NaN,2023-03-30,Thursday
117,TXN_2148617,Juice,3.0,3.0,9.0,Digital Wallet,NaN,2023-01-10,Tuesday


In [9]:
mask2 = data['Price Per Unit'].isna() & data['Total Spent'].notna() & data['Quantity'].notna() & (data['Quantity'] > 0)
data.loc[mask2, 'Price Per Unit'] = (data.loc[mask2, 'Total Spent'] / data.loc[mask2, 'Quantity']).round(2)

# Display the number of remaining NaN values in Price Per Unit
print(f"NaN values in Price Per Unit column after filling: {data['Price Per Unit'].isna().sum()}")

# Show a few examples of fixed rows
print("\nSample of rows where Price Per Unit was calculated:")
display(data[mask2].head())

NaN values in Price Per Unit column after filling: 32

Sample of rows where Price Per Unit was calculated:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Day of Week
56,TXN_3578141,Cake,5.0,3.0,15.0,NaN,Takeaway,2023-06-27,Tuesday
68,TXN_8427104,Salad,2.0,5.0,10.0,NaN,In-store,2023-10-27,Friday
85,TXN_8035512,Tea,3.0,1.5,4.5,Cash,NaN,2023-10-29,Sunday
104,TXN_7447872,Juice,2.0,3.0,6.0,NaN,NaN,NaT,NaN
140,TXN_2484241,Cake,3.0,3.0,9.0,Digital Wallet,NaN,2023-07-19,Wednesday


In [10]:
mask3 = data['Total Spent'].isna() & data['Price Per Unit'].notna() & data['Quantity'].notna()
data.loc[mask3, 'Total Spent'] = (data.loc[mask3, 'Price Per Unit'] * data.loc[mask3, 'Quantity']).round(2)

# Display the number of remaining NaN values in Total Spent
print(f"NaN values in Total Spent column after filling: {data['Total Spent'].isna().sum()}")

# Show a few examples of fixed rows
print("\nSample of rows where Total Spent was calculated:")
display(data[mask3].head())

NaN values in Total Spent column after filling: 37

Sample of rows where Total Spent was calculated:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Day of Week
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19,Wednesday
25,TXN_7958992,Smoothie,3.0,4.0,12.0,NaN,NaN,2023-12-13,Wednesday
42,TXN_6650263,Tea,2.0,1.5,3.0,NaN,Takeaway,2023-01-10,Tuesday
94,TXN_6289610,Juice,3.0,3.0,9.0,Cash,Takeaway,2023-08-07,Monday
143,TXN_8495063,Juice,1.0,3.0,3.0,Cash,NaN,2023-05-31,Wednesday


In [11]:
# Count NaN values in the three numeric columns for each row
nan_count = data[['Quantity', 'Price Per Unit', 'Total Spent']].isna().sum(axis=1)

# Identify rows with 2 or more NaN values in these columns
rows_to_drop = nan_count >= 2

# Drop these rows
data_cleaned = data[~rows_to_drop]

# Display the result
print(f"Original data shape: {data.shape}")
print(f"Cleaned data shape: {data_cleaned.shape}")
print(f"Number of rows dropped: {sum(rows_to_drop)}")


# Update the main dataframe
data = data_cleaned

Original data shape: (9031, 9)
Cleaned data shape: (8979, 9)
Number of rows dropped: 52


Kita juga akan mengdrop row yang memiliki transaksi datenya bernilai Null

In [12]:
# Drop rows where Transaction Date is null
data = data.dropna(subset=['Transaction Date'])

# Display the result
print(f"Shape before dropping null Transaction Date rows: {data_cleaned.shape}")
print(f"Shape after dropping null Transaction Date rows: {data.shape}")
print(f"Number of rows dropped: {data_cleaned.shape[0] - data.shape[0]}")

# Display the first few rows of the cleaned dataset
data.head()

Shape before dropping null Transaction Date rows: (8979, 9)
Shape after dropping null Transaction Date rows: (8564, 9)
Number of rows dropped: 415


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Day of Week
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08,Friday
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16,Tuesday
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19,Wednesday
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27,Thursday
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11,Sunday


In [13]:
# Check for duplicate records
duplicate_records = data.duplicated()
print(f"Total number of duplicate records: {duplicate_records.sum()}")

# Display the duplicate records if any exist
if duplicate_records.sum() > 0:
    print("\nDuplicate records:")
    print(data[data.duplicated(keep='first')])
    
    # Check duplicates based on Transaction ID (which should be unique)
    duplicate_txn_ids = data.duplicated(subset=['Transaction ID'], keep='first')
    print(f"\nDuplicate Transaction IDs: {duplicate_txn_ids.sum()}")
    if duplicate_txn_ids.sum() > 0:
        print(data[duplicate_txn_ids])

Total number of duplicate records: 0


In [14]:
# Print unique values for specified columns
print("Unique Items:")
print(data['Item'].unique())
print("\nUnique Payment Methods:")
print(data['Payment Method'].unique())
print("\nUnique Locations:")
print(data['Location'].unique())
print("\nUnique Days of Week:")
print(data['Day of Week'].unique())


Unique Items:
['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Sandwich' 'Juice' 'Tea']

Unique Payment Methods:
['Credit Card' 'Cash' nan 'Digital Wallet']

Unique Locations:
['Takeaway' 'In-store' nan]

Unique Days of Week:
['Friday' 'Tuesday' 'Wednesday' 'Thursday' 'Sunday' 'Saturday' 'Monday']
['Friday' 'Tuesday' 'Wednesday' 'Thursday' 'Sunday' 'Saturday' 'Monday']


In [15]:
# Fill NaN values with 'Unknown' for categorical columns before one-hot encoding
data['Payment Method'] = data['Payment Method'].fillna('Unknown')
data['Location'] = data['Location'].fillna('Unknown')

# Create one-hot encoding for 'Item'
item_dummies = pd.get_dummies(data['Item'], prefix='Item')

# Create one-hot encoding for 'Payment Method'
payment_dummies = pd.get_dummies(data['Payment Method'], prefix='Payment')

# Create one-hot encoding for 'Location'
location_dummies = pd.get_dummies(data['Location'], prefix='Location')

# Concatenate the one-hot encoded features with the original dataframe
# excluding the original categorical columns to avoid duplication
data_encoded = pd.concat([
    data.drop(['Item', 'Payment Method', 'Location'], axis=1),
    item_dummies,
    payment_dummies,
    location_dummies
], axis=1)

# Display the shape of the new dataframe
print(f"Shape of original dataframe: {data.shape}")
print(f"Shape of encoded dataframe: {data_encoded.shape}")

# Display the first few rows of the encoded dataframe
data_encoded.head()

Shape of original dataframe: (8564, 9)
Shape of encoded dataframe: (8564, 21)


,Transaction ID,Quantity,Price Per Unit,Total Spent,Transaction Date,Day of Week,Item_Cake,Item_Coffee,Item_Cookie,Item_Juice,...,Item_Sandwich,Item_Smoothie,Item_Tea,Payment_Cash,Payment_Credit Card,Payment_Digital Wallet,Payment_Unknown,Location_In-store,Location_Takeaway,Location_Unknown
0,TXN_1961373,2.0,2.0,4.0,2023-09-08,Friday,False,True,False,False,...,False,False,False,False,True,False,False,False,True,False
1,TXN_4977031,4.0,3.0,12.0,2023-05-16,Tuesday,True,False,False,False,...,False,False,False,True,False,False,False,True,False,False
2,TXN_4271903,4.0,1.0,4.0,2023-07-19,Wednesday,False,False,True,False,...,False,False,False,False,True,False,False,True,False,False
3,TXN_7034554,2.0,5.0,10.0,2023-04-27,Thursday,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,TXN_3160411,2.0,2.0,4.0,2023-06-11,Sunday,False,True,False,False,...,False,False,False,False,False,True,False,True,False,False


In [16]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8564 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Transaction ID          8564 non-null   object        
 1   Quantity                8564 non-null   float64       
 2   Price Per Unit          8564 non-null   float64       
 3   Total Spent             8564 non-null   float64       
 4   Transaction Date        8564 non-null   datetime64[ns]
 5   Day of Week             8564 non-null   object        
 6   Item_Cake               8564 non-null   bool          
 7   Item_Coffee             8564 non-null   bool          
 8   Item_Cookie             8564 non-null   bool          
 9   Item_Juice              8564 non-null   bool          
 10  Item_Salad              8564 non-null   bool          
 11  Item_Sandwich           8564 non-null   bool          
 12  Item_Smoothie           8564 non-null   bool         

In [17]:
# Drop Transaction Date column from data_encoded
data_encoded = data_encoded.drop('Transaction Date', axis=1)

# Check the shape after dropping the column
print(f"Shape of data_encoded after dropping Transaction Date: {data_encoded.shape}")

# Display the first few rows of the modified dataframe
data_encoded.head()

Shape of data_encoded after dropping Transaction Date: (8564, 20)


,Transaction ID,Quantity,Price Per Unit,Total Spent,Day of Week,Item_Cake,Item_Coffee,Item_Cookie,Item_Juice,Item_Salad,Item_Sandwich,Item_Smoothie,Item_Tea,Payment_Cash,Payment_Credit Card,Payment_Digital Wallet,Payment_Unknown,Location_In-store,Location_Takeaway,Location_Unknown
0,TXN_1961373,2.0,2.0,4.0,Friday,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False
1,TXN_4977031,4.0,3.0,12.0,Tuesday,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False
2,TXN_4271903,4.0,1.0,4.0,Wednesday,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False
3,TXN_7034554,2.0,5.0,10.0,Thursday,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True
4,TXN_3160411,2.0,2.0,4.0,Sunday,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False


In [18]:
# Convert Day of Week to a categorical code (0 for Monday, 1 for Tuesday, ..., 6 for Sunday)
day_mapping = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2, 
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

# Replace the Day of Week column with numeric codes
data_encoded['Day of Week'] = data_encoded['Day of Week'].map(day_mapping)

# Show the result
print("Day code mapping:", day_mapping)
print("\nData shape after converting days to codes:", data_encoded.shape)
data_encoded.head()


Day code mapping: {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

Data shape after converting days to codes: (8564, 20)


,Transaction ID,Quantity,Price Per Unit,Total Spent,Day of Week,Item_Cake,Item_Coffee,Item_Cookie,Item_Juice,Item_Salad,Item_Sandwich,Item_Smoothie,Item_Tea,Payment_Cash,Payment_Credit Card,Payment_Digital Wallet,Payment_Unknown,Location_In-store,Location_Takeaway,Location_Unknown
0,TXN_1961373,2.0,2.0,4.0,4,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False
1,TXN_4977031,4.0,3.0,12.0,1,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False
2,TXN_4271903,4.0,1.0,4.0,2,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False
3,TXN_7034554,2.0,5.0,10.0,3,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True
4,TXN_3160411,2.0,2.0,4.0,6,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False


In [19]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8564 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Transaction ID          8564 non-null   object 
 1   Quantity                8564 non-null   float64
 2   Price Per Unit          8564 non-null   float64
 3   Total Spent             8564 non-null   float64
 4   Day of Week             8564 non-null   int64  
 5   Item_Cake               8564 non-null   bool   
 6   Item_Coffee             8564 non-null   bool   
 7   Item_Cookie             8564 non-null   bool   
 8   Item_Juice              8564 non-null   bool   
 9   Item_Salad              8564 non-null   bool   
 10  Item_Sandwich           8564 non-null   bool   
 11  Item_Smoothie           8564 non-null   bool   
 12  Item_Tea                8564 non-null   bool   
 13  Payment_Cash            8564 non-null   bool   
 14  Payment_Credit Card     8564 non-null   bool 

In [20]:
data_encodeds = data_encoded.drop(['Transaction ID','Quantity', 'Total Spent'], axis=1) ## Drop Transaction ID and Total Spent for training
target = data_encoded['Quantity'] ## Target variable

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Inisialisasi model
model = RandomForestRegressor(random_state=42)

# Cross-validation
scores = cross_val_score(model, data_encodeds, target, cv=5, scoring='neg_mean_squared_error')
# Convert negative MSE to positive RMSE
rmse_scores = np.sqrt(-scores)
# Print RMSE scores for each fold
print("RMSE scores for each fold:")
print(rmse_scores)
# Print the mean RMSE score
print(f"Mean RMSE score: {rmse_scores.mean():.2f}")

RMSE scores for each fold:
[1.52060371 1.48185514 1.4839929  1.48114878 1.50777266]
Mean RMSE score: 1.50
